In [2]:
import random

random.seed(14)

In [3]:
import time

NOTEBOOK_START_TIME = time.time()

In [15]:
import gc
import math
import re
import sys
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from numpy.random import choice
PRIVATE = True

In [ ]:
#!pip install -U transformers --upgrade

In [6]:
import torch
import gc
torch.backends.cuda.enable_mem_efficient_sdp(False)

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    AutoConfig,
    StoppingCriteria,
    StoppingCriteriaList,
    set_seed
)

import transformers
print(f"Transformers Version: {transformers.__version__}")
set_seed(14)

Transformers Version: 4.42.3


In [7]:
MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/3"#"/kaggle/input/gemma/transformers/7b-it/1"
DEEP = True

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map='auto',
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
    )

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map='auto',
    )

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
gc.collect()

27

In [14]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]
        
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            last_token = input_ids[0][-len(stop):]
            if torch.all(torch.eq(stop,last_token)):
                return True
        return False


stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,  
stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [10]:
USE_PAST_KEY = True

temperature = 0.90
top_p = 1.0

temperature_coding = 0.90
top_p_coding = 1.0


In [35]:
fd=open('/kaggle/input/mathtext/problems2.txt','r').read()

In [36]:
fd[0:50]

'\nThere are 100 apples in a basket. If we remove 10'

In [37]:
fr=fd.split('\n')

In [38]:
len(fr)

425

In [39]:
fr[0:10]

['',
 'There are 100 apples in a basket. If we remove 10 apples from the basket, then how many apples will be left?',
 '',
 'A certain sum of money was divided among Amal, Kamal, and John. Amal got 25% more than Kamal got. John got 50% more than the total amount got by Amal and Kamal. If Kamal got $10, calculate how much money did John get. If 25% of a number is 30, what is the number?',
 '',
 '',
 'A rectangle has a length of 6 units and a width of 4 units. What is the perimeter of the rectangle?',
 '',
 'A boat can carry 400 kilograms. If the boat is carrying an empty barrel with a weight of 20 kilograms and a full barrel with a weight of 40 kilograms, what is the maximum number of barrels the boat can carry?',
 '']

In [40]:
lis=[]
for i in range(len(fr)):
    if len(fr[i])>10:
        lis.append(fr[i])

In [41]:
len(lis)

206

In [44]:
lis[2]

'A rectangle has a length of 6 units and a width of 4 units. What is the perimeter of the rectangle?'

In [57]:
pro = """
Below is a math problem you are to solve:\nThe problem is\n
\"{}\"
\nTo accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be number, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:
"""

In [16]:
pro

'Below, the observations X are:\n"{}"\n\n\ncluster these observations into k number of groups. First, apply bayesian information creteria to determine the value of K. Second, apply the mixture of normal distribution to cluster the observations into k cluster. Third, estimate all parameters and output the estimated observation membership.\n\n\n\nApproach:'

In [47]:
torch.cuda.empty_cache()
gc.collect()

12199

In [48]:
from IPython.display import Markdown as md

In [58]:
k=len(lis)
sol=[]
for i in range (0,k):
    problem=lis[i]
    initail_message = pro.format(problem,"{}")
    prompt = f"User: {initail_message}"
    print(prompt)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_len = len(model_inputs['input_ids'][0])
    generation_output = model.generate(**model_inputs,
                                       max_new_tokens=2048,
                                       return_dict_in_generate=USE_PAST_KEY,
                                       do_sample = True,
                                       temperature = temperature,
                                       top_p = top_p,
                                       num_return_sequences=6)##, stopping_criteria = stopping_criteria
    output_ids = generation_output.sequences[0]
    decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
    print(decoded_output)

User: 
Below is a math problem you are to solve:
The problem is

"There are 100 apples in a basket. If we remove 10 apples from the basket, then how many apples will be left?"

To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be number, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \boxed{}.

Approach:

User: 
Below is a math problem you are to solve:
The problem is

"There are 100 apples in a basket. If we remove 10 apples from the basket, then how many apples will be left?"

To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in e

KeyboardInterrupt: 